In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import torch.utils.data as data
from torchvision import transforms

import os
import os.path
import sys
if('/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path):
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2
import numpy as np
from scipy import signal
from PIL import Image
import random
import matplotlib.pyplot as plt

# base class for dataloader vision.py


class VisionDataset(data.Dataset):
    _repr_indent = 4

    def __init__(self, root, transforms=None, transform=None, target_transform=None):
        if isinstance(root, torch._six.string_classes):
            root = os.path.expanduser(root)
        self.root = root

        has_transforms = transforms is not None
        has_separate_transform = transform is not None or target_transform is not None
        if has_transforms and has_separate_transform:
            raise ValueError("Only transforms or transform/target_transform can "
                             "be passed as argument")

        # for backwards-compatibility
        self.transform = transform
        self.target_transform = target_transform

        if has_separate_transform:
            transforms = StandardTransform(transform, target_transform)
        self.transforms = transforms

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __repr__(self):
        head = "Dataset " + self.__class__.__name__
        body = ["Number of datapoints: {}".format(self.__len__())]
        if self.root is not None:
            body.append("Root location: {}".format(self.root))
        body += self.extra_repr().splitlines()
        if hasattr(self, "transforms") and self.transforms is not None:
            body += [repr(self.transforms)]
        lines = [head] + [" " * self._repr_indent + line for line in body]
        return '\n'.join(lines)

    def _format_transform_repr(self, transform, head):
        lines = transform.__repr__().splitlines()
        return (["{}{}".format(head, lines[0])] +
                ["{}{}".format(" " * len(head), line) for line in lines[1:]])

    def extra_repr(self):
        return ""


class StandardTransform(object):
    def __init__(self, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform

    def __call__(self, input, target):
        if self.transform is not None:
            input = self.transform(input)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return input, target

    def _format_transform_repr(self, transform, head):
        lines = transform.__repr__().splitlines()
        return (["{}{}".format(head, lines[0])] +
                ["{}{}".format(" " * len(head), line) for line in lines[1:]])

    def __repr__(self):
        body = [self.__class__.__name__]
        if self.transform is not None:
            body += self._format_transform_repr(self.transform,
                                                "Transform: ")
        if self.target_transform is not None:
            body += self._format_transform_repr(self.target_transform,
                                                "Target transform: ")

        return '\n'.join(body)


FILE_EXTENSIONS = ('front_0.jpg','tactile.txt','label.txt','front_rgb.mp4','left_rgb.mp4', 'pos.txt')
TACTILE_MAX_MAGNITUDE = 75431.01

def get_params(img, output_size):
    """random crop input sequence of frames.
    Args:
        img (PIL Image): Image to be cropped.
        output_size (tuple): Expected output size of the crop.
    Returns:
        tuple: params (i, j, h, w) to be passed to ``crop`` for random crop.
    """
    t, h, w, c = img.shape
    th, tw = output_size
    if w == tw and h == th:
        return 0, 0, h, w

    i = random.randint(0, h - th) if h!=th else 0
    j = random.randint(0, w - tw) if w!=tw else 0
    return i, j, th, tw

def random_crop(imgs):
    i, j, h, w = get_params(a, (224,224))
    imgs = imgs[:, i:i+h, j:j+w, :]
    return imgs

def tactile_loader(path, frames):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    output =[] 
    TACTILE_TIME = 24
    UPSAMPLE_FREQ = 18
    up_samples=TACTILE_TIME*UPSAMPLE_FREQ
    with open(path, 'rb') as f:
        tactile_frame = pd.read_csv(f,delimiter=' ', header=None)
        tactile = tactile_frame.as_matrix()
        tactile = tactile.astype('float')
        tactile = signal.resample(tactile,up_samples)
        tactile[np.where(tactile<0.1)] = 0.  # remove all the negative samples with 0
        tactile = tactile/TACTILE_MAX_MAGNITUDE #normalize input in range of 0 to 1.
        for frame in frames:
            output.append(tactile[frame,:].reshape(4,4,1))
        return np.array(output)

def video_to_tensor(frames):
    """Convert a ``numpy.ndarray`` to tensor.
    Converts a numpy.ndarray (T x H x W x C)
    to a torch.FloatTensor of shape (C x T x H x W)

    Args:
         pic (numpy.ndarray): Video to be converted to tensor.
    Returns:
         Tensor: Converted video.
    """
    return torch.from_numpy(frames.transpose([3,0,1,2]))

def compress_image(cv_frame, size):
    """
    Args:   frame - cv image
            size - tuple of (W x H)
    """
    return cv2.resize(cv_frame, size)

def preprocess_image(cv_frame, size):
    """
    Args:   frame - cv image
            size - tuple of (W x H)
    """
    im = cv2.resize(cv_frame, size) #compress image
    im = np.array(Image.fromarray(cv2.cvtColor(im,cv2.COLOR_BGR2RGB))) #convert to RGB image
    im = im/255 #normalize
    im = (im - 0.5)/0.5
    return im
    
def vid_loader(path):
    """
    1. Sample 64 frames from the video at equal interval.
    2. Compress image from (640 x 480) to (320 x 240)
    3.
    3. Use Random crop to get (224 x 224)
    Args:
        dir (string): Root directory path.
    Returns:
        tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
    Ensures:
        No class is a subdirectory of another.
    """
    #print(path)
    vidcap = cv2.VideoCapture(path)
    success,image = vidcap.read()
    count = 0
    required_frames = 64
    frame_output = 5
    # 18x18 = 324  frames in total
    length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
#     print("length",length)
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
#     print("FPS", fps )
    duration = length / fps
#     print("duration",duration)

    images = []
    frames = []
    while success:
        if((count%5)==0 and (len(images)<required_frames)):
            image = preprocess_image(image, (320,240))
            images.append(image)
            frames.append(count)
        success,image = vidcap.read()
        count += 1
#     print("normal len : " ,len(images))
    missing_frames = required_frames - len(images)
#     print("missing frames : " ,missing_frames)
    if(missing_frames):
        remove_element = (missing_frames//frame_output)+1
        del images[-remove_element:]
        del frames[-remove_element:]
        vidcap = cv2.VideoCapture(path)
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, length-missing_frames-remove_element)
        success,image = vidcap.read()
        temp_count = 0
        while success:
            image = preprocess_image(image, (320,240))
            images.append(image)
            frames.append(length-missing_frames-remove_element+temp_count)
            success,image = vidcap.read()
            temp_count +=1
    while(len(images) is not required_frames):
        vidcap = cv2.VideoCapture(path)
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, length-1)
        success,image = vidcap.read()
        image = preprocess_image(image, (320,240))
        images.append(image)
        frames.append(length-1)
    return np.array(images),np.array(frames)

def preprocess_grayscale_image(cv_frame, size):
    """
    Args:   frame - cv image
            size - tuple of (W x H)
    """
    im = cv2.resize(cv_frame, size) #compress image
    im = im/255 #normalize
    im = (im - 0.5)/0.5
    return im

def flow_loader(path, frames):
    flow_x = []
    flow_y = []
    images = []
    images_x = []
    images_y = []
    count = 0
    required_frames = 64
    for img in os.listdir(path):
        if img.endswith('x.png'):
            flow_x.append(img)
        else:
            flow_y.append(img)
    flow_x = sorted(flow_x)
    flow_y = sorted(flow_y)
    for frame in frames:
        while True:
            if(frame==int(flow_x[count][6:10])):
                images_x.append(flow_x[count])
                images_y.append(flow_y[count])
                break
            else:
                count += 1
    for x,y in zip(images_x,images_y):
        img_x = cv2.imread(path + x, cv2.IMREAD_GRAYSCALE)
        img_x = preprocess_grayscale_image(img_x,(320,240))
        img_y = cv2.imread(path + y, cv2.IMREAD_GRAYSCALE)
        img_y = preprocess_grayscale_image(img_y,(320,240))
        images.append(np.stack((img_x,img_y)).transpose([1,2,0])) #output shape T x H x W x C
    if(len(images) is not required_frames):
        raise ValueError("missing flow frames")
    return np.array(images)
    
#     count = 0
#     required_frames = 64
#     frame_output = 5
#     length = len(flow_x)
#     print(flow_x)
#     flow_x_list = flow_x[::frame_output]
#     flow_y_list = flow_y[::frame_output]

#     missing_frames = required_frames - len(flow_x_list)
#     if(missing_frames>0):
#         remove_element = (missing_frames//frame_output)+1
#         del flow_x_list[-remove_element:]
#         del flow_y_list[-remove_element:]
#         missing_frames = required_frames - len(flow_x_list)
#         [flow_x_list.append(i) for i in flow_x[-missing_frames:]]
#         [flow_y_list.append(i) for i in flow_y[-missing_frames:]]
#     elif(missing_frames<0): #contains extra frames and need to remove
#         del flow_x_list[-abs(missing_frames):]
#         del flow_y_list[-abs(missing_frames):]
#     images = []    
#     for x,y in zip(flow_x_list,flow_y_list):
#         img_x = cv2.imread(path + x, cv2.IMREAD_GRAYSCALE)
#         img_x = preprocess_grayscale_image(img_x,(320,240))
#         img_y = cv2.imread(path + y, cv2.IMREAD_GRAYSCALE)
#         img_y = preprocess_grayscale_image(img_y,(320,240))
#         images.append(np.stack((img_x,img_y)).transpose([1,2,0])) #output shape T x H x W x C
#     return np.array(images)

def old_flow_loader(path):
    flow_x = []
    flow_y = []
    for img in os.listdir(path):
        if img.endswith('x.png'):
            flow_x.append(img)
        else:
            flow_y.append(img)
    flow_x = sorted(flow_x)
    flow_y = sorted(flow_y)
    count = 0
    required_frames = 64
    frame_output = 5
    length = len(flow_x)

    flow_x_list = flow_x[::frame_output]
    flow_y_list = flow_y[::frame_output]

    missing_frames = required_frames - len(flow_x_list)
    if(missing_frames>0):
        remove_element = (missing_frames//frame_output)+1
        del flow_x_list[-remove_element:]
        del flow_y_list[-remove_element:]
        missing_frames = required_frames - len(flow_x_list)
        [flow_x_list.append(i) for i in flow_x[-missing_frames:]]
        [flow_y_list.append(i) for i in flow_y[-missing_frames:]]
    elif(missing_frames<0): #contains extra frames and need to remove
        del flow_x_list[-abs(missing_frames):]
        del flow_y_list[-abs(missing_frames):]
    images = []    
    for x,y in zip(flow_x_list,flow_y_list):
        img_x = cv2.imread(path + x, cv2.IMREAD_GRAYSCALE)
        img_x = preprocess_grayscale_image(img_x,(320,240))
        img_y = cv2.imread(path + y, cv2.IMREAD_GRAYSCALE)
        img_y = preprocess_grayscale_image(img_y,(320,240))
        images.append(np.stack((img_x,img_y)).transpose([1,2,0])) #output shape T x H x W x C
    return np.array(images)
"""
Make dataset with following input arguments:
split_file : .txt file containing path to each modalities to be trained
root: dataset folder path
mode: decides how many samples we need in the output Ex: "RGB" or "V", "FLOW" or "F", "VF",
        "VFT", "VFTP" (video, flow, tactile, position)
"""
def make_dataset(split_file, root, mode, transform):
    dataset = []
    file_path = np.loadtxt(split_file,dtype=str)
    prefix = root.split(file_path[0][:32])[0]
    file_names = ['front_rgb.mp4','left_rgb.mp4','tactile.txt','pos.txt','label.txt','flow']
#     print(prefix+file_path[0]+file_names[0])
    for i,file in enumerate(file_path):
        front_video_path = prefix + file + file_names[0]
        left_video_path = prefix + file + file_names[1]
        tactile_path = prefix + file + file_names[2]
        pos_path = prefix + file + file_names[3]
        label_path = prefix + file + file_names[4]
        front_flow_path = prefix + file + file_names[5] + '/' + 'front_rgb/'
        left_flow_path = prefix + file + file_names[5] + '/' + 'left_rgb/'
        dataset.append((front_video_path, left_video_path, tactile_path, pos_path, \
                       front_flow_path, left_flow_path, np.loadtxt(label_path)[3]))
    return dataset



class VisualTactile(VisionDataset):
    """Dataloader customized to load Visual-Tactile dataset
    Args:
        split_file (string): path to .txt file where we have path to each test and train object.
        root (string): Root directory path.
        mode (string) : For future implementation. (Decide which data to load V, F, VF, VFT, VFTP)
        transform (callable, optional): A function/transform that takes in
            a sample and returns a transformed version.
            E.g, ``transforms.RandomCrop`` for images.
     Attributes:
        samples (list): List of (sample path, class_index) tuples
        targets (list): The class_index value for each image in the dataset
    """

    def __init__(self, split_file, root, mode=None, transform=None):
        super(VisualTactile, self).__init__(root)

        self.root = root
        self.mode = mode
        self.split_file = split_file
        self.transform = transform
        self.extensions = FILE_EXTENSIONS

        # classes = name of the folder
        # class_to_idx = dictionary with class name(folder name) and index
        """classes, class_to_idx = self._find_classes(self.root)"""
        self.samples = make_dataset(self.split_file, self.root, self.mode, self.transform)
#         self.samples = old_make_dataset(self.root, class_to_idx, extensions, is_valid_file)
#         print("Samaples" , self.samples[0])
        if len(self.samples) == 0:
            raise (RuntimeError("Found 0 files in subfolders of: " + self.root + "\n"
                                "Supported extensions are: " + ",".join(extensions)))

        """self.classes = classes
        self.class_to_idx = class_to_idx"""
        self.targets = [s[-1] for s in self.samples]

    def _find_classes(self, dir):
        """
        Finds the class folders in a dataset.
        Args:
            dir (string): Root directory path.
        Returns:
            tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
        Ensures:
            No class is a subdirectory of another.
        """
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (f_video, l_video, tactile, pos, f_flow, l_flow, target) where target is successfull pickup or not.
        """
        #add conditoin here wich will return only the requested mode. Ex. V, or VFTP
        f_video, l_video, tactile, pos, f_flow, l_flow, target = self.samples[index]
        sample_f_video, selected_frames = vid_loader(f_video)
#         sample_l_video = vid_loader(l_video)
        sample_tactile = tactile_loader(tactile, selected_frames)
#         sample_pos = tactile_loader(pos)
        sample_target = target
#         sample_f_flow = flow_loader(f_flow, selected_frames)
#         sample_l_flow = flow_loader(l_flow)
        if self.transform is not None:
            sample_f_video = self.transform(sample_f_video)
            sample_l_video = self.transform(sample_l_video)
#         return sample_f_video, sample_l_video, sample_tactile, sample_pos, sample_target
#         return video_to_tensor(sample_f_video), video_to_tensor(sample_l_video), \
#                 torch.from_numpy(sample_tactile), torch.from_numpy(sample_pos), \
#                 video_to_tensor(sample_f_flow), video_to_tensor(sample_l_flow), \
#                 torch.from_numpy(np.array(sample_target))
#         return video_to_tensor(sample_f_video), video_to_tensor(sample_l_video), \
#                 torch.from_numpy(sample_tactile), torch.from_numpy(sample_pos), \
#                 torch.from_numpy(np.array(sample_target))
        return video_to_tensor(sample_tactile), sample_target

    def __len__(self):
        return len(self.samples)


class VisualTactileFolder(VisualTactile):
    """A generic data loader where the images are arranged in this way: ::
        root/dog/xxx.png
        root/dog/xxy.png
        root/dog/xxz.png
        root/cat/123.png
        root/cat/nsdf3.png
        root/cat/asd932_.png
    Args:
        root (string): Root directory path.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.
        is_valid_file (callable, optional): A function that takes path of an Image file
            and check if the file is a valid_file (used to check of corrupt files)
     Attributes:
        classes (list): List of the class names.
        class_to_idx (dict): Dict with items (class_name, class_index).
        imgs (list): List of (image path, class_index) tuples
    """

    def __init__(self, split_file, root, mode=None, transform=None):
        super(VisualTactileFolder, self).__init__(split_file, root, mode=None, transform=None)
        self.imgs = self.samples


In [ ]:
root = '/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/'
ds = VisualTactile('../master_i3d/testv2.txt', root)
ds_train = VisualTactile('../master_i3d/trainv2.txt', root)

In [ ]:
ds_train.samples[0]

In [ ]:
ds[0][0].shape

# 2. No upsampling, new tactile data folder, normalized by channel

In [11]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import torch.utils.data as data
from torchvision import transforms

import os
import os.path
import sys
if('/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path):
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2
import numpy as np
from scipy import signal
from PIL import Image
import random
import matplotlib.pyplot as plt

# base class for dataloader vision.py


class VisionDataset(data.Dataset):
    _repr_indent = 4

    def __init__(self, root, transforms=None, transform=None, target_transform=None):
        if isinstance(root, torch._six.string_classes):
            root = os.path.expanduser(root)
        self.root = root

        has_transforms = transforms is not None
        has_separate_transform = transform is not None or target_transform is not None
        if has_transforms and has_separate_transform:
            raise ValueError("Only transforms or transform/target_transform can "
                             "be passed as argument")

        # for backwards-compatibility
        self.transform = transform
        self.target_transform = target_transform

        if has_separate_transform:
            transforms = StandardTransform(transform, target_transform)
        self.transforms = transforms

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __repr__(self):
        head = "Dataset " + self.__class__.__name__
        body = ["Number of datapoints: {}".format(self.__len__())]
        if self.root is not None:
            body.append("Root location: {}".format(self.root))
        body += self.extra_repr().splitlines()
        if hasattr(self, "transforms") and self.transforms is not None:
            body += [repr(self.transforms)]
        lines = [head] + [" " * self._repr_indent + line for line in body]
        return '\n'.join(lines)

    def _format_transform_repr(self, transform, head):
        lines = transform.__repr__().splitlines()
        return (["{}{}".format(head, lines[0])] +
                ["{}{}".format(" " * len(head), line) for line in lines[1:]])

    def extra_repr(self):
        return ""


class StandardTransform(object):
    def __init__(self, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform

    def __call__(self, input, target):
        if self.transform is not None:
            input = self.transform(input)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return input, target

    def _format_transform_repr(self, transform, head):
        lines = transform.__repr__().splitlines()
        return (["{}{}".format(head, lines[0])] +
                ["{}{}".format(" " * len(head), line) for line in lines[1:]])

    def __repr__(self):
        body = [self.__class__.__name__]
        if self.transform is not None:
            body += self._format_transform_repr(self.transform,
                                                "Transform: ")
        if self.target_transform is not None:
            body += self._format_transform_repr(self.target_transform,
                                                "Target transform: ")

        return '\n'.join(body)


FILE_EXTENSIONS = ('front_0.jpg','tactile.txt','label.txt','front_rgb.mp4','left_rgb.mp4', 'pos.txt')
TACTILE_MAX_MAGNITUDE = np.array([23655, 20662, 14496,  6475, 41133, 64793, 59317, 33177, 19897,
       62084, 49874, 29170, 42944, 14976, 12311, 14331])

def get_params(img, output_size):
    """random crop input sequence of frames.
    Args:
        img (PIL Image): Image to be cropped.
        output_size (tuple): Expected output size of the crop.
    Returns:
        tuple: params (i, j, h, w) to be passed to ``crop`` for random crop.
    """
    t, h, w, c = img.shape
    th, tw = output_size
    if w == tw and h == th:
        return 0, 0, h, w

    i = random.randint(0, h - th) if h!=th else 0
    j = random.randint(0, w - tw) if w!=tw else 0
    return i, j, th, tw

def random_crop(imgs):
    i, j, h, w = get_params(a, (224,224))
    imgs = imgs[:, i:i+h, j:j+w, :]
    return imgs

def tactile_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    output =[]
    groups = []
    tactile_frame = np.loadtxt(path)
    tactile = tactile_frame.astype('float')
#     tactile = tactile/TACTILE_MAX_MAGNITUDE
#     #group of 4 sensors
#     n=4
#     out = [tactile[:,k:k+n] for k in range(0, tactile.shape[1], n)]
#     for data in out:
#         for i in range(data.shape[1]):
#             groups.append(tactile[:324,i].reshape(18,18,1))
#         temp = video_to_tensor(np.array(groups))
#         output.append(temp)
#         groups = []
    return tactile

def video_to_tensor(frames):
    """Convert a ``numpy.ndarray`` to tensor.
    Converts a numpy.ndarray (T x H x W x C)
    to a torch.FloatTensor of shape (C x T x H x W)

    Args:
         pic (numpy.ndarray): Video to be converted to tensor.
    Returns:
         Tensor: Converted video.
    """
    return torch.from_numpy(frames.transpose([3,0,1,2]))

def compress_image(cv_frame, size):
    """
    Args:   frame - cv image
            size - tuple of (W x H)
    """
    return cv2.resize(cv_frame, size)

def preprocess_image(cv_frame, size):
    """
    Args:   frame - cv image
            size - tuple of (W x H)
    """
    im = cv2.resize(cv_frame, size) #compress image
    im = np.array(Image.fromarray(cv2.cvtColor(im,cv2.COLOR_BGR2RGB))) #convert to RGB image
    im = im/255 #normalize
    im = (im - 0.5)/0.5
    return im
    
def vid_loader(path):
    """
    1. Sample 64 frames from the video at equal interval.
    2. Compress image from (640 x 480) to (320 x 240)
    3.
    3. Use Random crop to get (224 x 224)
    Args:
        dir (string): Root directory path.
    Returns:
        tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
    Ensures:
        No class is a subdirectory of another.
    """
    #print(path)
    vidcap = cv2.VideoCapture(path)
    success,image = vidcap.read()
    count = 0
    required_frames = 64
    frame_output = 5
    # 18x18 = 324  frames in total
    length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
#     print("length",length)
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
#     print("FPS", fps )
    duration = length / fps
#     print("duration",duration)

    images = []
    frames = []
    while success:
        if((count%5)==0 and (len(images)<required_frames)):
            image = preprocess_image(image, (320,240))
            images.append(image)
            frames.append(count)
        success,image = vidcap.read()
        count += 1
#     print("normal len : " ,len(images))
    missing_frames = required_frames - len(images)
#     print("missing frames : " ,missing_frames)
    if(missing_frames):
        remove_element = (missing_frames//frame_output)+1
        del images[-remove_element:]
        del frames[-remove_element:]
        vidcap = cv2.VideoCapture(path)
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, length-missing_frames-remove_element)
        success,image = vidcap.read()
        temp_count = 0
        while success:
            image = preprocess_image(image, (320,240))
            images.append(image)
            frames.append(length-missing_frames-remove_element+temp_count)
            success,image = vidcap.read()
            temp_count +=1
    while(len(images) is not required_frames):
        vidcap = cv2.VideoCapture(path)
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, length-1)
        success,image = vidcap.read()
        image = preprocess_image(image, (320,240))
        images.append(image)
        frames.append(length-1)
    return np.array(images),np.array(frames)

def preprocess_grayscale_image(cv_frame, size):
    """
    Args:   frame - cv image
            size - tuple of (W x H)
    """
    im = cv2.resize(cv_frame, size) #compress image
    im = im/255 #normalize
    im = (im - 0.5)/0.5
    return im

def flow_loader(path, frames):
    flow_x = []
    flow_y = []
    images = []
    images_x = []
    images_y = []
    count = 0
    required_frames = 64
    for img in os.listdir(path):
        if img.endswith('x.png'):
            flow_x.append(img)
        else:
            flow_y.append(img)
    flow_x = sorted(flow_x)
    flow_y = sorted(flow_y)
    for frame in frames:
        while True:
            if(frame==int(flow_x[count][6:10])):
                images_x.append(flow_x[count])
                images_y.append(flow_y[count])
                break
            else:
                count += 1
    for x,y in zip(images_x,images_y):
        img_x = cv2.imread(path + x, cv2.IMREAD_GRAYSCALE)
        img_x = preprocess_grayscale_image(img_x,(320,240))
        img_y = cv2.imread(path + y, cv2.IMREAD_GRAYSCALE)
        img_y = preprocess_grayscale_image(img_y,(320,240))
        images.append(np.stack((img_x,img_y)).transpose([1,2,0])) #output shape T x H x W x C
    if(len(images) is not required_frames):
        raise ValueError("missing flow frames")
    return np.array(images)
    
#     count = 0
#     required_frames = 64
#     frame_output = 5
#     length = len(flow_x)
#     print(flow_x)
#     flow_x_list = flow_x[::frame_output]
#     flow_y_list = flow_y[::frame_output]

#     missing_frames = required_frames - len(flow_x_list)
#     if(missing_frames>0):
#         remove_element = (missing_frames//frame_output)+1
#         del flow_x_list[-remove_element:]
#         del flow_y_list[-remove_element:]
#         missing_frames = required_frames - len(flow_x_list)
#         [flow_x_list.append(i) for i in flow_x[-missing_frames:]]
#         [flow_y_list.append(i) for i in flow_y[-missing_frames:]]
#     elif(missing_frames<0): #contains extra frames and need to remove
#         del flow_x_list[-abs(missing_frames):]
#         del flow_y_list[-abs(missing_frames):]
#     images = []    
#     for x,y in zip(flow_x_list,flow_y_list):
#         img_x = cv2.imread(path + x, cv2.IMREAD_GRAYSCALE)
#         img_x = preprocess_grayscale_image(img_x,(320,240))
#         img_y = cv2.imread(path + y, cv2.IMREAD_GRAYSCALE)
#         img_y = preprocess_grayscale_image(img_y,(320,240))
#         images.append(np.stack((img_x,img_y)).transpose([1,2,0])) #output shape T x H x W x C
#     return np.array(images)

def old_flow_loader(path):
    flow_x = []
    flow_y = []
    for img in os.listdir(path):
        if img.endswith('x.png'):
            flow_x.append(img)
        else:
            flow_y.append(img)
    flow_x = sorted(flow_x)
    flow_y = sorted(flow_y)
    count = 0
    required_frames = 64
    frame_output = 5
    length = len(flow_x)

    flow_x_list = flow_x[::frame_output]
    flow_y_list = flow_y[::frame_output]

    missing_frames = required_frames - len(flow_x_list)
    if(missing_frames>0):
        remove_element = (missing_frames//frame_output)+1
        del flow_x_list[-remove_element:]
        del flow_y_list[-remove_element:]
        missing_frames = required_frames - len(flow_x_list)
        [flow_x_list.append(i) for i in flow_x[-missing_frames:]]
        [flow_y_list.append(i) for i in flow_y[-missing_frames:]]
    elif(missing_frames<0): #contains extra frames and need to remove
        del flow_x_list[-abs(missing_frames):]
        del flow_y_list[-abs(missing_frames):]
    images = []    
    for x,y in zip(flow_x_list,flow_y_list):
        img_x = cv2.imread(path + x, cv2.IMREAD_GRAYSCALE)
        img_x = preprocess_grayscale_image(img_x,(320,240))
        img_y = cv2.imread(path + y, cv2.IMREAD_GRAYSCALE)
        img_y = preprocess_grayscale_image(img_y,(320,240))
        images.append(np.stack((img_x,img_y)).transpose([1,2,0])) #output shape T x H x W x C
    return np.array(images)
"""
Make dataset with following input arguments:
split_file : .txt file containing path to each modalities to be trained
root: dataset folder path
mode: decides how many samples we need in the output Ex: "RGB" or "V", "FLOW" or "F", "VF",
        "VFT", "VFTP" (video, flow, tactile, position)
"""
def make_dataset(split_file, root, mode, transform):
    dataset = []
    file_path = np.loadtxt(split_file,dtype=str)
    prefix = root.split(file_path[0][:32])[0]
    file_names = ['front_rgb.mp4','left_rgb.mp4','tactile.txt','pos.txt','label.txt','flow']
    for i,file in enumerate(file_path):
        front_video_path = prefix + file + file_names[0]
        left_video_path = prefix + file + file_names[1]
        tactile_path = prefix +"/Visual-Tactile_Dataset/tactile_data/"+file.split(file[:32])[1] + file_names[2]
        pos_path = prefix + file + file_names[3]
        label_path = prefix + file + file_names[4]
        front_flow_path = prefix + file + file_names[5] + '/' + 'front_rgb/'
        left_flow_path = prefix + file + file_names[5] + '/' + 'left_rgb/'
        dataset.append((front_video_path, left_video_path, tactile_path, pos_path, \
                       front_flow_path, left_flow_path, np.loadtxt(label_path)[3]))
    return dataset



class VisualTactile(VisionDataset):
    """Dataloader customized to load Visual-Tactile dataset
    Args:
        split_file (string): path to .txt file where we have path to each test and train object.
        root (string): Root directory path.
        mode (string) : For future implementation. (Decide which data to load V, F, VF, VFT, VFTP)
        transform (callable, optional): A function/transform that takes in
            a sample and returns a transformed version.
            E.g, ``transforms.RandomCrop`` for images.
     Attributes:
        samples (list): List of (sample path, class_index) tuples
        targets (list): The class_index value for each image in the dataset
    """

    def __init__(self, split_file, root, mode=None, transform=None):
        super(VisualTactile, self).__init__(root)

        self.root = root
        self.mode = mode
        self.split_file = split_file
        self.transform = transform
        self.extensions = FILE_EXTENSIONS

        # classes = name of the folder
        # class_to_idx = dictionary with class name(folder name) and index
        """classes, class_to_idx = self._find_classes(self.root)"""
        self.samples = make_dataset(self.split_file, self.root, self.mode, self.transform)
#         self.samples = old_make_dataset(self.root, class_to_idx, extensions, is_valid_file)
#         print("Samaples" , self.samples[0])
        if len(self.samples) == 0:
            raise (RuntimeError("Found 0 files in subfolders of: " + self.root + "\n"
                                "Supported extensions are: " + ",".join(extensions)))

        """self.classes = classes
        self.class_to_idx = class_to_idx"""
        self.targets = [s[-1] for s in self.samples]

    def _find_classes(self, dir):
        """
        Finds the class folders in a dataset.
        Args:
            dir (string): Root directory path.
        Returns:
            tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
        Ensures:
            No class is a subdirectory of another.
        """
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (f_video, l_video, tactile, pos, f_flow, l_flow, target) where target is successfull pickup or not.
        """
        #add conditoin here wich will return only the requested mode. Ex. V, or VFTP
        f_video, l_video, tactile, pos, f_flow, l_flow, target = self.samples[index]
#         sample_f_video, selected_frames = vid_loader(f_video)
#         sample_l_video = vid_loader(l_video)
        sample_tactile = tactile_loader(tactile)
#         sample_pos = tactile_loader(pos)
        sample_target = target
#         sample_f_flow = flow_loader(f_flow, selected_frames)
#         sample_l_flow = flow_loader(l_flow)
        if self.transform is not None:
            sample_f_video = self.transform(sample_f_video)
            sample_l_video = self.transform(sample_l_video)
#         return sample_f_video, sample_l_video, sample_tactile, sample_pos, sample_target
#         return video_to_tensor(sample_f_video), video_to_tensor(sample_l_video), \
#                 torch.from_numpy(sample_tactile), torch.from_numpy(sample_pos), \
#                 video_to_tensor(sample_f_flow), video_to_tensor(sample_l_flow), \
#                 torch.from_numpy(np.array(sample_target))
#         return video_to_tensor(sample_f_video), video_to_tensor(sample_l_video), \
#                 torch.from_numpy(sample_tactile), torch.from_numpy(sample_pos), \
#                 torch.from_numpy(np.array(sample_target))
        return sample_tactile, sample_target

    def __len__(self):
        return len(self.samples)


class VisualTactileFolder(VisualTactile):
    """A generic data loader where the images are arranged in this way: ::
        root/dog/xxx.png
        root/dog/xxy.png
        root/dog/xxz.png
        root/cat/123.png
        root/cat/nsdf3.png
        root/cat/asd932_.png
    Args:
        root (string): Root directory path.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.
        is_valid_file (callable, optional): A function that takes path of an Image file
            and check if the file is a valid_file (used to check of corrupt files)
     Attributes:
        classes (list): List of the class names.
        class_to_idx (dict): Dict with items (class_name, class_index).
        imgs (list): List of (image path, class_index) tuples
    """

    def __init__(self, split_file, root, mode=None, transform=None):
        super(VisualTactileFolder, self).__init__(split_file, root, mode=None, transform=None)
        self.imgs = self.samples


In [19]:
root = '/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/'
ds = VisualTactile('../master_i3d/trainv2.txt', root)

In [ ]:
# ds.samples[0]

In [15]:
tactile = ds[0][0]
ds[10][0]

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,  88.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0., 126., ...,   0., 221.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [ ]:
s = 'frame_0000_x.png'
int(s[6:10])

In [ ]:
folder = VisualTactileFolder('test.txt',root)
tactile_path = folder.samples[0][2]

In [ ]:
tactile_path

In [ ]:
sr = np.array(tactile[7]).reshape(4,-1)
plt.imshow(sr)

In [ ]:
norm = (sr - sr.mean())/np.std(sr)
# plt.imshow(norm)
np.linalg.norm(norm)

# fixing bug in flowloader

In [ ]:
def vid_loader(path):
    """
    1. Sample 64 frames from the video at equal interval.
    2. Compress image from (640 x 480) to (320 x 240)
    3.
    3. Use Random crop to get (224 x 224)
    Args:
        dir (string): Root directory path.
    Returns:
        tuple: (classes, class_to_idx) where classes are relative to (dir), and class_to_idx is a dictionary.
    Ensures:
        No class is a subdirectory of another.
    """
    #print(path)
    vidcap = cv2.VideoCapture(path)
    success,image = vidcap.read()
    count = 0
    required_frames = 64
    frame_output = 5
    # 18x18 = 324  frames in total
    length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
#     print("length",length)
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
#     print("FPS", fps )
    duration = length / fps
#     print("duration",duration)

    images = []
    frames = []
    while success:
        if((count%5)==0 and (len(images)<required_frames)):
            image = preprocess_image(image, (320,240))
            images.append(image)
            frames.append(count)
        success,image = vidcap.read()
        count += 1
#     print("normal len : " ,len(images))
    missing_frames = required_frames - len(images)
#     print("missing frames : " ,missing_frames)
    if(missing_frames):
        remove_element = (missing_frames//frame_output)+1
        del images[-remove_element:]
        del frames[-remove_element:]
        vidcap = cv2.VideoCapture(path)
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, length-missing_frames-remove_element)
        success,image = vidcap.read()
        temp_count = 0
        while success:
            image = preprocess_image(image, (320,240))
            images.append(image)
            frames.append(length-missing_frames-remove_element+temp_count)
            success,image = vidcap.read()
            temp_count +=1
    while(len(images) is not required_frames):
        vidcap = cv2.VideoCapture(path)
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, length-1)
        success,image = vidcap.read()
        image = preprocess_image(image, (320,240))
        images.append(image)
        frames.append(length-1)
    return np.array(images),np.array(frames)

def preprocess_grayscale_image(cv_frame, size):
    """
    Args:   frame - cv image
            size - tuple of (W x H)
    """
    im = cv2.resize(cv_frame, size) #compress image
    im = im/255 #normalize
    im = (im - 0.5)/0.5
    return im

def flow_loader(path, frames):
    flow_x = []
    flow_y = []
    images = []
    images_x = []
    images_y = []
    count = 0
    required_frames = 64
    for img in os.listdir(path):
        if img.endswith('x.png'):
            flow_x.append(img)
        else:
            flow_y.append(img)
    flow_x = sorted(flow_x)
    flow_y = sorted(flow_y)
    for frame in frames:
        #bug fixed by count=frame
        count = frame
        while True:
            if(frame==int(flow_x[count][6:10])):
                images_x.append(flow_x[count])
                images_y.append(flow_y[count])
                break
            else:
                count += 1
    print(images_x)
    for x,y in zip(images_x,images_y):
        img_x = cv2.imread(path + x, cv2.IMREAD_GRAYSCALE)
        img_x = preprocess_grayscale_image(img_x,(320,240))
        img_y = cv2.imread(path + y, cv2.IMREAD_GRAYSCALE)
        img_y = preprocess_grayscale_image(img_y,(320,240))
        images.append(np.stack((img_x,img_y)).transpose([1,2,0])) #output shape T x H x W x C
    if(len(images) != required_frames):
        raise ValueError("missing flow frames")
    return np.array(images)

In [ ]:
#test your errorneous file here
vpath = "/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Coffeecup/50_470/top/7/front_rgb.mp4"
fpath = "/home/pritesh/Desktop/front_rgb/"
images, frames = vid_loader(vpath)
flow = flow_loader(fpath, frames)

In [ ]:
len(flow)
# frames

# Image normalization

In [ ]:
im = np.array(ds[0][0])
im

In [ ]:
tim = im.transpose([1,2,3,0])
# normalized = (tim[0] - 0.5)/0.5
plt.imshow(tim[0])

In [ ]:
import PIL.Image as Image
x = np.array(Image.open('../../../../../frame_0052_x.png'))
y = np.array(Image.open('../../../../../frame_0052_y.png'))
flow = np.stack((x,y))
nx = preprocess_grayscale_image(y, (320,240))
plt.imshow(nx,cmap='gray')

In [ ]:
plt.imshow(y,cmap='gray')

# Max of tactile and normalize with this

In [16]:
#finding maximum from the test set
max_list = []
for data in ds:
    max_list.append(data[0].max())
max_list_train = []
for data in ds_train:
    max_list.append(data[0].max())

NameError: name 'ds_train' is not defined

In [17]:
datapt = len(ds.samples)
max_tactile_test = np.arange(16)
for i,data in enumerate(ds):
    a = np.max(data[0],axis=0)
    for j in range(16):
        if(a[j]>max_tactile_test[j]):
            max_tactile_test[j]=a[j]
    sys.stdout.write("\rstatus {} % completed".format(((i+1)/datapt)*100))
    sys.stdout.flush()

status 100.0 % completed % completeddd

In [20]:
datapt = len(ds.samples)
max_tactile_train = np.arange(16)
for i,data in enumerate(ds):
    a = np.max(data[0],axis=0)
    for j in range(16):
        if(a[j]>max_tactile_train[j]):
            max_tactile_train[j]=a[j]
    sys.stdout.write("\rstatus {} % completed".format(((i+1)/datapt)*100))
    sys.stdout.flush()

status 100.0 % completed % completeddd

In [22]:
# TACTILE_MAX_MAGNITUDE = np.array([23655, 20662, 14496,  6475, 41133, 64793, 59317, 33177, 19897,
#        62084, 49874, 29170, 42944, 14976, 12311, 14331])
max_tactile_test
max_tactile_train
TACTILE_MAX = np.max(np.stack((max_tactile_test,max_tactile_train)), axis=0)
TACTILE_MAX

array([23655, 20662, 14496,  6475, 41133, 64793, 59317, 33177, 19897,
       62084, 49874, 29170, 42944, 14976, 12311, 14331])

In [18]:
max_tactile_test

array([19844, 20007, 13876,  2212, 34104, 23571,  3778,  9039, 15113,
       55713, 44345, 20870, 27789,  7034,  6753, 14331])

In [21]:
max_tactile_train

array([23655, 20662, 14496,  6475, 41133, 64793, 59317, 33177, 19897,
       62084, 49874, 29170, 42944, 14976, 12311, 14125])

In [ ]:
tactile/TACTILE_MAX

In [ ]:

for i in range(10):
    sys.stdout.write("\rDoing thing {}".format(i))
    sys.stdout.flush()

In [ ]:
max(max_list)

# Tactile image visualization

In [ ]:
reshaped_tactile = tactile[:324,15].reshape(18,18)
plt.imshow(reshaped_tactile)
plt.figure(figsize=(10,5))
for i in range(16):
    plt.plot(tactile[:,i])

# tactile net v1

In [ ]:
class tactileNet(nn.Module):
    def __init__(self):
        super(tactileNet,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv3d(1, 16, kernel_size=(2,3,3), padding=1),nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv3d(16, 32, kernel_size=(2,3,3), padding=0),nn.ReLU(),nn.MaxPool3d(kernel_size=(2,2,2)))
        self.fc1 = nn.Sequential(nn.Linear(1024, 64, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
        self.fc2 = nn.Linear(64,1)
    def forward(self, x):
        out = self.layer1(x)
#        print(out.shape)
        out = self.layer2(out)
#        print(out.shape)
        out = out.view(out.size(0), -1)
#        print(out.shape)
        out = self.fc1(out)
#        print(out.shape)
        out = self.fc2(out)
#        print(out.shape)
        return out

# tactile net v2

In [74]:
import torch.nn as nn
class tactileNet(nn.Module):
    def __init__(self):
        super(tactileNet,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv3d(1, 16, kernel_size=(2,3,3), padding=1),nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv3d(16, 32, kernel_size=(2,3,3), padding=0),nn.ReLU(),nn.MaxPool3d(kernel_size=(2,4,4)))
        self.fc1 = nn.Sequential(nn.Linear(1024, 64, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
#         self.fc2 = nn.Linear(64,1)
    def forward(self, x):
        out = self.layer1(x)
#         print(out.shape)
        out = self.layer2(out)
#         print(out.shape)
        out = out.view(out.size(0), -1)
#         print(out.shape)
        out = self.fc1(out)
#         print(out.shape)
        return out

class tactileFusionNet(nn.Module):
    def __init__(self, tac_model1, tac_model2, tac_model3, tac_model4):
        super(tactileFusionNet,self).__init__()
        self.tac_model1 = tac_model1
        self.tac_model2 = tac_model2
        self.tac_model3 = tac_model3
        self.tac_model4 = tac_model4
        self.fc1 = nn.Sequential(nn.Linear(64*4, 128, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
        self.fc2 = nn.Linear(128,1)
    def forward(self, x1,x2,x3,x4):
        t1 = self.tac_model1(x1)
        print(t1.shape)
        t2 = self.tac_model2(x2)
        t3 = self.tac_model3(x3)
        t4 = self.tac_model4(x4)
        out = torch.cat((t1,t2,t3,t4),dim=1)
        out = self.fc1(out)
#         print(out.shape)
        out = self.fc2(out)
#         print(out.shape)
        return out

In [79]:
path = "/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Coffeecup/50_843/back/0/tactile.txt"
a = tactile_loader(path)
fuseNet1 = tactileNet()
fuseNet2 = tactileNet()
fuseNet3 = tactileNet()
fuseNet4 = tactileNet()
masterNet = tactileFusionNet(fuseNet1,fuseNet2,fuseNet3,fuseNet4)
masterNet(a[0].unsqueeze(0).float(),a[1].unsqueeze(0).float(),a[2].unsqueeze(0).float(),a[3].unsqueeze(0).float())
# inp = a[0].unsqueeze(0)
# fuseNet(inp.float())

torch.Size([1, 64])


tensor([[-0.0612]], grad_fn=<AddmmBackward>)

# loading trained model

In [ ]:
import tensorboardX
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms


import numpy as np

# from charades_dataset import Charades as Dataset
from visual_tactile_dataset import VisualTactile as Dataset
import cv2
import matplotlib.pyplot as plt

class tactileNet(nn.Module):
    def __init__(self):
        super(tactileNet,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv3d(1, 16, kernel_size=(2,3,3), padding=1),nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv3d(16, 32, kernel_size=(2,3,3), padding=0),nn.ReLU(),nn.MaxPool3d(kernel_size=(2,2,2)))
        self.fc1 = nn.Sequential(nn.Linear(1024, 64, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
        self.fc2 = nn.Linear(64,1)
    def forward(self, x):
        out = self.layer1(x)
#        print(out.shape)
        out = self.layer2(out)
#        print(out.shape)
        out = out.view(out.size(0), -1)
#        print(out.shape)
        out = self.fc1(out)
#        print(out.shape)
        out = self.fc2(out)
#        print(out.shape)
        return out

net = tactileNet()
net = nn.DataParallel(net)
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.0000001)
lr_sched = optim.lr_scheduler.MultiStepLR(optimizer, [50, 100, 150, 200])
data = torch.load('model_31.tar',map_location='cpu')
net.load_state_dict(data['model_state'])
optimizer.load_state_dict(data['optimizer_state'])
lr_sched.load_state_dict(data['scheduler_state'])

def run(init_lr=0.001, max_steps=200, mode='rgb', root='/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/',\
        train_split='train.txt', test_split='test.txt', batch_size=1, save_model=''):
    writer = tensorboardX.SummaryWriter()
    # setup dataset

    dataset = Dataset(train_split, root, mode)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

    val_dataset = Dataset(test_split, root, mode)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

    dataloaders = {'train': dataloader, 'val': val_dataloader}
    datasets = {'train': dataset, 'val': val_dataset}
    
    steps = 0
    with open('i3d_video.txt', 'w') as file: 
        file.write("train and validation loss file\n")
    # train it
    print ('Step {}/{}'.format(steps, max_steps))
    print ('-' * 10)

    # Each epoch has a training and validation phase
    net.train(False)  # Set model to evaluate mode

    tot_cls_loss = 0.0
    num_iter = 0
    count = 0
    optimizer.zero_grad()
    # Iterate over data.
    phase = 'val'
    for data in dataloaders[phase]:
        num_iter += 1
        # get the inputs
        tactile, labels = data

        if torch.cuda.is_available():
            tact_input = Variable(tactile.cuda())
            t = tact_input.size(2)
            labels = Variable(labels.cuda())
        else:
            tact_input = Variable(tactile)
            t = tact_input.size(2)
            labels = Variable(labels)

        out = net(tact_input.float())
        #print('prediction output = ', per_frame_logits.shape)
        #print('labels = ',labels.shape)
        # compute classification loss (with max-pooling along time B x C x T)
        out = out.squeeze(1)
        cls_loss = F.binary_cross_entropy_with_logits(out.double(), labels.double())
#         print(cls_loss,labels)
        tot_cls_loss += cls_loss.item()
#         cls_loss.backward()
        print('{}. {} Loss: {:.4f} and lr: {}'.format(num_iter, phase,tot_cls_loss/num_iter,init_lr))
        with open('i3d_video.txt', 'a') as file: 
            file.write("%f\n" %(tot_cls_loss/num_iter))
#         optimizer.step()
#         optimizer.zero_grad()
        if phase == 'val':
            writer.add_scalar('error/'+ phase , (tot_cls_loss/num_iter), num_iter)
        else:
            writer.add_scalar('error/'+ phase, (tot_cls_loss/num_iter), num_iter)
    #save error at every epoch
    writer.add_scalar('errorAtEpoch/'+phase, (tot_cls_loss/num_iter), steps)
    tot_cls_loss = 0.
#if(steps%50 == 0):
#    torch.save(fusedNet.module.state_dict(), save_model+phase+str(steps).zfill(6)+'.pt')
#    save_checkpoint(fusedNet, optimizer, lr_sched, steps)

run()

In [ ]:
index = np.where(a<1)
a[index] = 0
np.max(a)

In [ ]:
a = tactile[0]

plt.figure(figsize=(15,7))
time = np.linspace(0,24,len(a))
for i in range(1,17):
    plt.plot(time,a[:,i-1],label=(i-1))
    plt.legend()
#     plt.axvline(label[0]/10, color="green")
#     plt.axvline(label[1]/10, color="yellow")
#     plt.axvline(label[2]/10, color="r")
    plt.title("Cola 150-26 right-8")
plt.show()
plt.figure(figsize=(15,7))
for i in range(1,17):
    plt.plot(np.linspace(0,24,len(td)),td[:,i-1],label=(i-1))

In [ ]:
import torch.nn as nn
class tactileNet(nn.Module):
    def __init__(self):
        super(tactileNet,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv3d(1, 16, kernel_size=(2,3,3), padding=1),nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv3d(16, 32, kernel_size=(2,3,3), padding=0),nn.ReLU(),nn.MaxPool3d(kernel_size=(2,2,2)))
        self.fc1 = nn.Sequential(nn.Linear(1024, 64, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
        self.fc2 = nn.Linear(64,1)
    def forward(self, x):
        out = self.layer1(x)
        print(out.shape)
        out = self.layer2(out)
        print(out.shape)
        out = out.view(out.size(0), -1)
        print(out.shape)
        out = self.fc1(out)
        print(out.shape)
        out = self.fc2(out)
        print(out.shape)
        return out

In [ ]:
net = tactileNet()

In [ ]:
a = tactile[0][4,:].reshape(4,4,1)
torch.save()

In [ ]:
def get_input(data):
    output = []
    frames = np.random.randint(0,431,64)
    for frame in frames:
        output.append(data[frame,:].reshape(4,4,1))
    return np.array(output)
    
inp = get_input(tactile[0])
inp = video_to_tensor(inp)
inp = inp.unsqueeze(0)
inp.shape
# inp = torch.from_numpy(inp)

In [ ]:
output = net(inp.float())
torch.save(net.state_dict(), 'ok.pt')

In [ ]:
from scipy import interpolate
y = np.array(range(400))
x = np.array(range(16))
a = td
xx, yy = np.meshgrid(x, y)
f = interpolate.interp2d(x, y, a, kind='linear')

# xnew = np.linspace(0, 16, 4)
# ynew = np.linspace(0, 24*18, 4)
xnew = np.arange(16)
ynew = np.linspace(0, 400, (24*18))
znew = f(xnew, ynew)
plt.figure(figsize=(15,8))
for i in range(1,17):
    plt.semilogy(np.linspace(0,24,len(td)),znew[:400,i-1],label=(i-1))
    plt.legend()
for i in range(1,17):
    plt.semilogy(np.linspace(0,24,len(td)),td[:,i-1],label=(i-1))
    plt.legend()

In [ ]:
# import time
# for i in range(len(tactile[0])):
# #     plt.imshow(tactile[0][i,:].reshape(4,4),cmap='gray')
#     cv2.imshow('im',tactile[0][i,:].reshape(4,4))
#     cv2.waitKey(0)
    
cv2.destroyAllWindows()

In [ ]:
import torch.optim as optim
optim.Adam()

In [ ]:
b = np.arange(16)
a = np.max(tactile,axis=0)
for i in range(16):
    if(a[i]>b[i]):
        b[i]=a[i]

In [12]:
import numpy as np
TACTILE_MAX_MAGNITUDE = np.array([23655, 20662, 14496,  6475, 41133, 64793, 59317, 33177, 19897,
       62084, 49874, 29170, 42944, 14976, 12311, 14331])
def tactile_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    output =[]
    groups = []
    tactile_frame = np.loadtxt(path)
    tactile = tactile_frame.astype('float')
    tactile = tactile/TACTILE_MAX_MAGNITUDE
    #group of 4 sensors
    n=4
    out = [tactile[:,k:k+n] for k in range(0, tactile.shape[1], n)]
    for data in out:
        for i in range(data.shape[1]):
            groups.append(tactile[:324,i].reshape(18,18,1))
        temp = video_to_tensor(np.array(groups))
        output.append(temp)
        groups = []
    return output

In [60]:
import torch.nn as nn
class tactileNet(nn.Module):
    def __init__(self):
        super(tactileNet,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv3d(1, 16, kernel_size=(2,3,3), padding=1),nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv3d(16, 32, kernel_size=(2,3,3), padding=0),nn.ReLU(),nn.MaxPool3d(kernel_size=(2,4,4)))
        self.fc1 = nn.Sequential(nn.Linear(1024, 64, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
#         self.fc2 = nn.Linear(64,1)
    def forward(self, x):
        out = self.layer1(x)
#         print(out.shape)
        out = self.layer2(out)
#         print(out.shape)
        out = out.view(out.size(0), -1)
#         print(out.shape)
        out = self.fc1(out)
#         print(out.shape)
        return out

class tactileFusionNet(nn.Module):
    def __init__(self, tac_model1, tac_model2, tac_model3, tac_model4):
        super(tactileNet,self).__init__()
        self.tac_model1 = tac_model1
        self.tac_model2 = tac_model2
        self.tac_model3 = tac_model3
        self.tac_model4 = tac_model4
        self.fc1 = nn.Sequential(nn.Linear(64*4, 128, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
        self.fc2 = nn.Linear(128,1)
    def forward(self, x1,x2,x3,x4):
        t1 = self.tac_model1(x1)
        t2 = self.tac_model2(x2)
        t3 = self.tac_model3(x3)
        t4 = self.tac_model4(x4)
        out = torch.cat((t1,t2,t3,t4),dim=1)
        out = self.fc1(out)
#         print(out.shape)
        out = self.fc2(out)
#         print(out.shape)
        return out

In [59]:
path = "/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Coffeecup/50_843/back/0/tactile.txt"
a = tactile_loader(path)
fuseNet = tactileNet()
inp = a[0].unsqueeze(0)
fuseNet(inp.float())

torch.Size([1, 16, 5, 18, 18])
torch.Size([1, 32, 2, 4, 4])
torch.Size([1, 1024])
torch.Size([1, 64])
torch.Size([1, 1])


tensor([[0.0734]], grad_fn=<AddmmBackward>)

## Reference

* https://stackoverflow.com/questions/51420923/resampling-a-signal-with-scipy-signal-resample
* Why normalization is required in neural network: https://visualstudiomagazine.com/articles/2014/01/01/how-to-standardize-data-for-neural-networks.aspx